In [1]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import scienceplots
plt.style.use(['science', 'grid'])
mpl.rcParams["font.size"] = "12"

import torch
import torch.nn.init as init

from modules.utils import FeedForwardNetwork, plot_ode, plot_losses, rmse
from modules.problems import LorenzSystem

In [2]:
def train(
    problem,
    model,
    wi, wx, wy, wz, num_iters, N_D, lr,
    collect_every=1000, print_every=1000
):
    collocation_points = torch.linspace(0, problem.T, N_D, requires_grad=True).reshape(-1, 1)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    losses, errors = [], []

    for i in range(num_iters + 1):

        optimizer.zero_grad()

        L_I = problem.loss_initial(model)
        L_X, L_Y, L_Z = problem.loss_physical(model, collocation_points)

        L = wi * L_I + wx * L_X + wy * L_Y + wz * L_Z

        L.backward()
        optimizer.step()
        
        if i % collect_every == 0:
            preds = model(problem.t.reshape(-1, 1)).detach().numpy()
            x = preds[:,0].flatten()
            y = preds[:,1].flatten()
            z = preds[:,2].flatten()

            error_x = rmse(x, problem.solution[0])
            error_y = rmse(y, problem.solution[1])
            error_z = rmse(z, problem.solution[2])

            losses.append(np.array([L_I.item(), L_X.item(), L_Y.item(), L_Z.item(), L.item()]))
            errors.append(np.array([error_x, error_y, error_z]))
        
        if i % print_every == 0:
            print(f'Iteration {i} --- {error_x}, {error_y}, {error_z}')
    
    return np.array(losses), np.array(errors)

In [ ]:
T = 1
s, p, b = 10, 28, 8/3
x_0, y_0, z_0 = 15, 8, 12
test_points = 1500
problem = LorenzSystem(T, (s, p, b), [x_0, y_0, z_0], test_points)

plot_ode(
    problem.t.numpy(), 
    [
        (problem.solution[0], r'$x(t)$', None), 
        (problem.solution[1], r'$y(t)$', None),
        (problem.solution[2], r'$z(t)$', None)
    ],
    size=(5, 3))

In [ ]:
L, W = 2, 64
model = FeedForwardNetwork(L, W, input_dim=1, output_dim=3)
model.init_weights(init.normal_)

wi, wx, wy, wz = 1, 1, 1, 1
num_iters = 2500
N_D = 1024*6
lr = 1e-3
collect_every = 250

losses, errors = train(
    problem,
    model,
    wi, wx, wy, wz, num_iters, N_D, lr,
    collect_every=collect_every, print_every=500
)

In [ ]:
preds = model(problem.t.reshape(-1, 1)).detach().numpy()
x = preds[:,0].flatten()
y = preds[:,1].flatten()
z = preds[:,2].flatten()

plot_ode(
    problem.t.numpy(), 
    solutions=[
        (problem.solution[0], 'Solution', 'b'), 
        (problem.solution[1], None, 'b'),
        (problem.solution[2], None, 'b'),
        ],
    predicted=[
        (x, r'$\mathcal{N}(t)$', 'r'), 
        (y, None, 'r'),
        (z, None, 'r')
        ],
    title='Lorenz System', size=(5, 3)
)

In [ ]:
ax = plt.figure(figsize=(6, 6)).add_subplot(projection='3d')

ax.plot(xs=problem.solution.T[:,0], ys=problem.solution.T[:,1], zs=problem.solution.T[:,2], lw=1)
ax.plot(*preds.T, lw=1)
ax.set_xlabel("X Axis")
ax.set_ylabel("Y Axis")
ax.set_zlabel("Z Axis")
ax.set_title("Lorenz Attractor")

plt.show()

In [ ]:
plot_losses(
    np.arange(0, num_iters+1, collect_every),
    losses=[
        (losses[:,0], r'$\mathcal{L_I}$'),
        (losses[:,1], r'$\mathcal{L_X}$'),
        (losses[:,2], r'$\mathcal{L_Y}$'),
        (losses[:,2], r'$\mathcal{L_Z}$'),
        (losses[:,3], r'$\mathcal{L_T}$')
    ],
    errors=[
        (errors[:,0], r'$\mathcal{X}(t)$'),
        (errors[:,1], r'$\mathcal{Y}(t)$'),
        (errors[:,2], r'$\mathcal{Z}(t)$')
    ]
)